In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'

# Read Data

In [2]:
def file_reader(basedir, filename):
    if filename.endswith('.csv'):
        try:
            df = pd.read_csv(basedir + filename, header=15)
        except:
            df = pd.read_csv(basedir + filename, encoding='euc-kr', header=15)
    elif filename.endswith('.xlsx'):
        try:
            df = pd.read_excel(basedir + filename, header=16)
        except:
            df = pd.read_excel(basedir + filename, encoding='euc-kr', header=16)
    else:
        print('error')
    
    return df

In [3]:
def read_data():
    basedir = './국토교통부_실거래가_공개시스템/'
    filenames = os.listdir(basedir)
    
    apart_trade_files = [filename for filename in filenames if filename.startswith('아파트(매매)')]
    apart_lease_files = [filename for filename in filenames if filename.startswith('아파트(전월세)')]
    multiplex_trade_files = [filename for filename in filenames if filename.startswith('연립다세대(매매)')]
    multiplex_lease_files = [filename for filename in filenames if filename.startswith('연립다세대(전월세)')]
    officetel_trade_files = [filename for filename in filenames if filename.startswith('오피스텔(매매)')]
    officetel_lease_files = [filename for filename in filenames if filename.startswith('오피스텔(전월세)')]
    
    #
    apart_trade_dfs_list = []
    for filename in apart_trade_files:
        df = file_reader(basedir, filename)
        apart_trade_dfs_list.append(df)
    apart_trade_df = pd.concat(apart_trade_dfs_list).reset_index(drop=True)
    
    #
    apart_lease_dfs_list = []
    for filename in apart_lease_files:
        df = file_reader(basedir, filename)
        apart_lease_dfs_list.append(df)
    apart_lease_df = pd.concat(apart_lease_dfs_list).reset_index(drop=True)
    
    #
    multiplex_trade_dfs_list = []
    for filename in multiplex_trade_files:
        df = file_reader(basedir, filename)
        multiplex_trade_dfs_list.append(df)
    multiplex_trade_df = pd.concat(multiplex_trade_dfs_list).reset_index(drop=True)
    
    #
    multiplex_lease_dfs_list = []
    for filename in multiplex_lease_files:
        df = file_reader(basedir, filename)
        multiplex_lease_dfs_list.append(df)
    multiplex_lease_df = pd.concat(multiplex_lease_dfs_list).reset_index(drop=True)
    
    #
    officetel_trade_dfs_list = []
    for filename in officetel_trade_files:
        df = file_reader(basedir, filename)
        officetel_trade_dfs_list.append(df)
    officetel_trade_df = pd.concat(officetel_trade_dfs_list).reset_index(drop=True)
    
    #
    officetel_lease_dfs_list = []
    for filename in officetel_lease_files:
        df = file_reader(basedir, filename)
        officetel_lease_dfs_list.append(df)
    officetel_lease_df = pd.concat(officetel_lease_dfs_list).reset_index(drop=True)
    
    
    return apart_trade_df, apart_lease_df, multiplex_trade_df, multiplex_lease_df, officetel_trade_df, officetel_lease_df

In [4]:
%%time
apart_trade_df, apart_lease_df, multiplex_trade_df, multiplex_lease_df, officetel_trade_df, officetel_lease_df\
= read_data()

C:\Users\나\AppData\Roaming\Python\Python37\site-packages\IPython\core\magics\execution.py:1321: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)


Wall time: 4min 24s


In [5]:
apart_trade_df.head()

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,"57,000",2,1988,언주로 103
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,"57,000",2,1988,언주로 103
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,"55,000",5,1987,언주로 3
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,22,"58,250",4,1987,언주로 3
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201305,10,"60,000",5,1987,언주로 3


In [5]:
def bon_bun_allocator(x):
    splitted = x.split('-')
    return int(splitted[0])

def bu_bun_allocator(x):
    splitted = x.split('-')
    if len(splitted) == 2:
        return int(splitted[1])
    elif len(splitted) == 1:
        return 0
    else:
        print('error')
        return

In [6]:
def create_land_prices_df():
    df = pd.read_csv('AL_11_D151_20200925.csv', encoding='euc-kr', usecols=[2,5,6,8])
    
    df = df[df['기준년도'] >= 2006].reset_index(drop=True)
    
    simple_cols = ['법정동명', '지번', '기준년도', '공시지가']
    
    bonbun = df['지번'].apply(bon_bun_allocator)
    bubun = df['지번'].apply(bu_bun_allocator)
    df['전체주소'] = df['법정동명'] + ' ' + bonbun.apply(str) + '-' + bubun.apply(str)
    
    df = df[['전체주소', '기준년도', '공시지가']]
    
    df['공시지가'] = df['공시지가'] / 10000
    
    df.columns = ['전체주소', '년', '공시지가(만원)']
    
    return df

In [7]:
%%time
land_prices_df = create_land_prices_df()

Wall time: 3min 56s


In [8]:
def land_plans_full_address_allocator(df):    
    bonbun = df['지번'].apply(bon_bun_allocator)
    bubun = df['지번'].apply(bu_bun_allocator)
    
    df['전체주소'] = df['법정동명'] + ' ' + bonbun.apply(str) + '-' + bubun.apply(str)
    
    simple_cols = ['전체주소', '년', '용도지역지구명']
    df = df[simple_cols]
    
    return df.drop_duplicates()

In [9]:
def land_plan_unstack(df):
    df['이진항'] = 1
    df = df.set_index(['전체주소', '년', '용도지역지구명'])
    df = df.unstack(-1, 0).reset_index()
    
    cols = df.columns
    new_cols = []
    for i in range(len(cols)):
        if i == 0:
            new_cols.append(cols[i][0])
        else:
            new_cols.append(cols[i][1])
    
    df.columns = new_cols
    
    return df

In [10]:
def create_land_plans_df():
    df = pd.read_csv('./토지이용계획정보/AL_11_D155_20201107.csv', encoding='euc-kr').drop_duplicates()
    df['년'] = df['등록일자'].str.split('-').apply(lambda x: x[0]).astype('int')
    
    df = land_plans_full_address_allocator(df)    
    df = land_plan_unstack(df)
    
    return df

In [11]:
#land_plans_df = create_land_plans_df()

In [12]:
def landnum_modifier(x):
    # x is a string
    
    splitted = x.split('-')
    if len(splitted) == 1:
        return x + '-0'
    else:
        return x

In [13]:
def create_land_specs_df():
    df = pd.read_csv('./토지특성정보/AL_11_D195_20200925.csv', encoding='euc-kr', usecols=[2, 5, 7, 10, 13, 19, 21, 23])
    
    df.fillna('지정되지않음', inplace=True)
    
    df['전체주소'] = df['법정동명'] + ' ' + df['지번'].apply(landnum_modifier)
    
    categorical_cols = ['지목명', '용도지역명1', '지형높이', '지형형상', '도로접면']
    
    for col in tqdm(categorical_cols, position=0):
        unique_values = df[col].unique()
        for v in unique_values:
            df[col + '_' + '{}'.format(v)] = 0
            df[col + '_' + '{}'.format(v)] = df[col].apply(lambda x: 1 if x == v else 0)
    
    df.drop(columns=categorical_cols + ['법정동명', '지번'], inplace=True)
    
    return df

In [14]:
land_specs_df = create_land_specs_df()

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:34<00:00, 30.85s/it]


# From Previous EDA & Preprocessing

In [15]:
apart_lease_df.drop(apart_lease_df[apart_lease_df['전용면적(㎡)'].isna()].index, inplace=True)

In [16]:
# 전용면적과 층 null들을 median 값으로 교체 시 사용

def area_floor_fillna_median(df):
    df = df.copy()
    
    df['temp_full_addr'] = df['시군구'].str.split(' ').apply(lambda x: x[0]) + ' ' + df['시군구'].str.split(' ').apply(lambda x: x[1])\
                           + ' ' + df['도로명'] + ' ' + df['단지명']
    
    area_null_df = df[df['전용면적(㎡)'].isna()]
    
    unique_addresses = area_null_df['temp_full_addr'].unique()
    
    for address in unique_addresses:
        addr_df = df[df['temp_full_addr'] == address]
        
        area_median = addr_df['전용면적(㎡)'].median()
        
        df.loc[addr_df[addr_df['전용면적(㎡)'].isna()].index, '전용면적(㎡)'] = area_median
        
    df = df.drop(df[df['전용면적(㎡)'].isna()].index)
        
        
    floor_null_df = df[df['층'].isna()]
    
    unique_addresses = floor_null_df['temp_full_addr'].unique()
    
    for address in unique_addresses:
        addr_df = df[df['temp_full_addr'] == address]
        
        floor_median = addr_df['층'].median()
        
        df.loc[addr_df[addr_df['층'].isna()].index, '층'] = floor_median
        
    df = df.drop(df[df['층'].isna()].index)
        
    return df

In [17]:
multiplex_trade_df.loc[multiplex_trade_df[multiplex_trade_df['건축년도'].isna()].index, '건축년도'] = 1971
multiplex_lease_df.drop(multiplex_lease_df[multiplex_lease_df['층'].isna()].index, inplace=True)

In [18]:
# 모든 df 도로명 빈 칸 null 로 교체

def replace_empty_to_null(df):
    df = df.copy()
    
    empty_df = df[df['도로명'] == ' ']
    
    df.loc[empty_df.index, '도로명'] = np.nan
    
    return df

In [19]:
apart_trade_df = replace_empty_to_null(apart_trade_df)
apart_lease_df = replace_empty_to_null(apart_lease_df)
multiplex_trade_df = replace_empty_to_null(multiplex_trade_df)
multiplex_lease_df = replace_empty_to_null(multiplex_lease_df)
officetel_trade_df = replace_empty_to_null(officetel_trade_df)
officetel_lease_df = replace_empty_to_null(officetel_lease_df)

In [20]:
multiplex_lease_df.loc[multiplex_lease_df[multiplex_lease_df['건축년도'].isna()].index, '건축년도'] = 1971

In [21]:
# 건축년도가 null 인 row 들에 있는 주소들 중에서, 같은 주소지만 건축년도 값이 있는 주소는 null 을 채워넣고, 없는 주소는 df 에서 제거.
# 추가적으로, 한 주소지, 단지이름에 건축년도가 두 가지 이상 있는지도 조사함.

def bltyear_fill_and_remove_null(df):
    df = df.copy()
    
    df['temp_full_addr'] = df['시군구'] + df['번지'] + df['단지명']
    
    bltyear_null_df = df[df['건축년도'].isna()]
    
    unique_addrs = bltyear_null_df['temp_full_addr'].unique()
    
    concat_list = []
    for addr in unique_addrs:
        addr_df = df[df['temp_full_addr'] == addr]
        if addr_df['건축년도'].isna().sum() != addr_df.shape[0]:
            if addr_df['건축년도'].std() != 0:
                print('건축년도가 두 가지 이상인 주소가 있음.')
            
            addr_df['건축년도'].fillna(addr_df['건축년도'].mean(), inplace=True)
            
            concat_list.append(addr_df)
    
    df.drop(columns=['temp_full_addr'], inplace=True)
    
    if len(concat_list) > 0:
        concat_df = pd.concat(concat_list)
    else:
        return df
    
    #return concat_df
    
    df.loc[concat_df.index, '건축년도'] = concat_df['건축년도']
    
    bltyear_null_df = df[df['건축년도'].isna()]
    
    df.drop(bltyear_null_df.index, inplace=True)
        
    return df.reset_index(drop=True)

In [22]:
officetel_trade_df = bltyear_fill_and_remove_null(officetel_trade_df)
officetel_lease_df = bltyear_fill_and_remove_null(officetel_lease_df)

In [23]:
def trade_dfs_change_dtype(df):
    df = df.copy()
    
    df['거래금액(만원)'] = df['거래금액(만원)'].apply(str)
    df['거래금액(만원)'] = pd.to_numeric(df['거래금액(만원)'].str.replace(',', ''))
    df['층'] = df['층'].astype('int')
    df['건축년도'] = df['건축년도'].astype('int')
    
    return df

In [24]:
apart_trade_df = trade_dfs_change_dtype(apart_trade_df)
multiplex_trade_df = trade_dfs_change_dtype(multiplex_trade_df)
officetel_trade_df = trade_dfs_change_dtype(officetel_trade_df)

In [25]:
def lease_dfs_change_dtype(df):
    df = df.copy()
    
    df['보증금(만원)'] = df['보증금(만원)'].apply(str)
    df['월세(만원)'] = df['월세(만원)'].apply(str)
    df['보증금(만원)'] = pd.to_numeric(df['보증금(만원)'].str.replace(',', ''))
    df['월세(만원)'] = pd.to_numeric(df['월세(만원)'].str.replace(',', ''))
    df['층'] = df['층'].astype('int')
    df['건축년도'] = df['건축년도'].astype('int')
    
    return df

In [26]:
apart_lease_df = lease_dfs_change_dtype(apart_lease_df)
multiplex_lease_df = lease_dfs_change_dtype(multiplex_lease_df)
officetel_lease_df = lease_dfs_change_dtype(officetel_lease_df)

In [27]:
def price_per_area_and_year_month(df, mode='trade'):
    df = df.copy()
    
    if mode == 'trade':
        df['가격/면적'] = df['거래금액(만원)'] / df['전용면적(㎡)']
    elif mode == 'lease':
        df['가격/면적'] = df['보증금(만원)'] / df['전용면적(㎡)']
    
    df['year'] = pd.to_numeric(df['계약년월'].astype('str').str[:4])
    df['month'] = pd.to_numeric(df['계약년월'].astype('str').str[4:])
    
    return df   

In [28]:
apart_trade_df = price_per_area_and_year_month(apart_trade_df)
multiplex_trade_df = price_per_area_and_year_month(multiplex_trade_df)
officetel_trade_df = price_per_area_and_year_month(officetel_trade_df)

In [29]:
apart_lease_df = price_per_area_and_year_month(apart_lease_df, mode='lease')
multiplex_lease_df = price_per_area_and_year_month(multiplex_lease_df, mode='lease')
officetel_lease_df = price_per_area_and_year_month(officetel_lease_df, mode='lease')

In [30]:
def si_gu(df):
    df = df.copy()
    
    df['시'] = df['시군구'].str.split(' ').apply(lambda x: x[0])
    df['구'] = df['시군구'].str.split(' ').apply(lambda x: x[1])
    
    return df

In [31]:
apart_trade_df = si_gu(apart_trade_df)
apart_lease_df = si_gu(apart_lease_df)
multiplex_trade_df = si_gu(multiplex_trade_df)
multiplex_lease_df = si_gu(multiplex_lease_df)
officetel_trade_df = si_gu(officetel_trade_df)
officetel_lease_df = si_gu(officetel_lease_df)

In [32]:
apart_lease_deposit_only_df = apart_lease_df[apart_lease_df['전월세구분'] == '전세'].reset_index(drop=True)
apart_lease_monthly_pay_df = apart_lease_df[apart_lease_df['전월세구분'] == '월세'].reset_index(drop=True)
multiplex_lease_deposit_only_df = multiplex_lease_df[multiplex_lease_df['전월세구분'] == '전세'].reset_index(drop=True)
multiplex_lease_monthly_pay_df = multiplex_lease_df[multiplex_lease_df['전월세구분'] == '월세'].reset_index(drop=True)
officetel_lease_deposit_only_df = officetel_lease_df[officetel_lease_df['전월세구분'] == '전세'].reset_index(drop=True)
officetel_lease_monthly_pay_df = officetel_lease_df[officetel_lease_df['전월세구분'] == '월세'].reset_index(drop=True)

In [33]:
# Tukey의 기법을 사용하여 이상치 (outlier) 검색 후 제거 (가격)

def find_and_remove_Tukey_outliers_price(df, mode='trade'):
    df = df.copy()
    # mode = 'trade' or 'lease'
    
    if mode == 'trade':
        df['target_transformed'] = quantile_transform(df[['거래금액(만원)']], output_distribution='normal', random_state=42)
    elif mode == 'lease':
        df['target_transformed'] = quantile_transform(df[['보증금(만원)']], output_distribution='normal', random_state=42)
    
    q1 = df['target_transformed'].quantile(0.25)
    q3 = df['target_transformed'].quantile(0.75)
    iqr = q3 - q1
    outlier_step = 2 * iqr

    outlier_df = df[(df['target_transformed'] < q1 - outlier_step)]# | (df['target_transformed'] > q3 + outlier_step)]
    
    
    print('{}개의 이상치를 찾아서 제거했습니다.'.format(outlier_df.shape[0]))
    
    df.drop(outlier_df.index, inplace=True)
    df.drop(columns=['target_transformed'], inplace=True)
    df = df.reset_index(drop=True)
    
    return df

In [34]:
apart_trade_df = find_and_remove_Tukey_outliers_price(apart_trade_df)
multiplex_trade_df = find_and_remove_Tukey_outliers_price(multiplex_trade_df)
officetel_trade_df = find_and_remove_Tukey_outliers_price(officetel_trade_df)

48개의 이상치를 찾아서 제거했습니다.
66개의 이상치를 찾아서 제거했습니다.
11개의 이상치를 찾아서 제거했습니다.


In [35]:
apart_lease_deposit_only_df = find_and_remove_Tukey_outliers_price(apart_lease_deposit_only_df, mode='lease')
apart_lease_monthly_pay_df = find_and_remove_Tukey_outliers_price(apart_lease_monthly_pay_df, mode='lease')
multiplex_lease_deposit_only_df = find_and_remove_Tukey_outliers_price(multiplex_lease_deposit_only_df, mode='lease')
multiplex_lease_monthly_pay_df = find_and_remove_Tukey_outliers_price(multiplex_lease_monthly_pay_df, mode='lease')
officetel_lease_deposit_only_df = find_and_remove_Tukey_outliers_price(officetel_lease_deposit_only_df, mode='lease')
officetel_lease_monthly_pay_df = find_and_remove_Tukey_outliers_price(officetel_lease_monthly_pay_df, mode='lease')

114개의 이상치를 찾아서 제거했습니다.
307개의 이상치를 찾아서 제거했습니다.
109개의 이상치를 찾아서 제거했습니다.
409개의 이상치를 찾아서 제거했습니다.
57개의 이상치를 찾아서 제거했습니다.
105개의 이상치를 찾아서 제거했습니다.


In [36]:
# Tukey의 기법을 사용하여 이상치 (outlier) 검색 후 제거 (가격/면적)

def find_and_remove_Tukey_outliers_price_per_area(df):
    df = df.copy()
    
    df['target_transformed'] = quantile_transform(df[['가격/면적']], output_distribution='normal', random_state=42)
    
    q1 = df['target_transformed'].quantile(0.25)
    q3 = df['target_transformed'].quantile(0.75)
    iqr = q3 - q1
    outlier_step = 1.81 * iqr

    outlier_df = df[(df['target_transformed'] < q1 - outlier_step) | (df['target_transformed'] > q3 + outlier_step)]
    
    
    print('{}개의 이상치를 찾아서 제거했습니다.'.format(outlier_df.shape[0]))
    
    df.drop(outlier_df.index, inplace=True)
    df.drop(columns=['target_transformed'], inplace=True)
    df = df.reset_index(drop=True)
    
    return df

In [37]:
apart_trade_df = find_and_remove_Tukey_outliers_price_per_area(apart_trade_df)
multiplex_trade_df = find_and_remove_Tukey_outliers_price_per_area(multiplex_trade_df)
officetel_trade_df = find_and_remove_Tukey_outliers_price_per_area(officetel_trade_df)

1178개의 이상치를 찾아서 제거했습니다.
898개의 이상치를 찾아서 제거했습니다.
217개의 이상치를 찾아서 제거했습니다.


In [38]:
def remove_zero_rows(df):
    df = df.copy()
    
    zero_df = df[df['전용면적(㎡)'] == 0]
    
    print('{}개의 0값을 찾아서 제거했습니다.'.format(zero_df.shape[0]))
    
    df.drop(zero_df.index, inplace=True)
    
    return df

In [39]:
apart_trade_df = remove_zero_rows(apart_trade_df)
multiplex_trade_df = remove_zero_rows(multiplex_trade_df)
officetel_trade_df = remove_zero_rows(officetel_trade_df)
apart_lease_deposit_only_df = remove_zero_rows(apart_lease_deposit_only_df)
apart_lease_monthly_pay_df = remove_zero_rows(apart_lease_monthly_pay_df)
multiplex_lease_deposit_only_df = remove_zero_rows(multiplex_lease_deposit_only_df)
multiplex_lease_monthly_pay_df = remove_zero_rows(multiplex_lease_monthly_pay_df)
officetel_lease_deposit_only_df = remove_zero_rows(officetel_lease_deposit_only_df)
officetel_lease_monthly_pay_df = remove_zero_rows(officetel_lease_monthly_pay_df)

0개의 0값을 찾아서 제거했습니다.
0개의 0값을 찾아서 제거했습니다.
0개의 0값을 찾아서 제거했습니다.
0개의 0값을 찾아서 제거했습니다.
0개의 0값을 찾아서 제거했습니다.
4개의 0값을 찾아서 제거했습니다.
5개의 0값을 찾아서 제거했습니다.
0개의 0값을 찾아서 제거했습니다.
0개의 0값을 찾아서 제거했습니다.


In [40]:
apart_lease_deposit_only_df = find_and_remove_Tukey_outliers_price_per_area(apart_lease_deposit_only_df)
apart_lease_montly_pay_df = find_and_remove_Tukey_outliers_price_per_area(apart_lease_monthly_pay_df)
multiplex_lease_deposit_only_df = find_and_remove_Tukey_outliers_price_per_area(multiplex_lease_deposit_only_df)
multiplex_lease_monthly_pay_df = find_and_remove_Tukey_outliers_price_per_area(multiplex_lease_monthly_pay_df)
officetel_lease_deposit_only_df = find_and_remove_Tukey_outliers_price_per_area(officetel_lease_deposit_only_df)
officetel_lease_monthly_pay_df = find_and_remove_Tukey_outliers_price_per_area(officetel_lease_monthly_pay_df)

1894개의 이상치를 찾아서 제거했습니다.
645개의 이상치를 찾아서 제거했습니다.
438개의 이상치를 찾아서 제거했습니다.
190개의 이상치를 찾아서 제거했습니다.
191개의 이상치를 찾아서 제거했습니다.
176개의 이상치를 찾아서 제거했습니다.


In [41]:
apart_trade_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전용면적(㎡)', '계약년월', '계약일', '가격(만원)',
                          '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시', '구']
multiplex_trade_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전용면적(㎡)', '대지권면적(㎡)', '계약년월', '계약일',
                              '가격(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시', '구']
officetel_trade_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전용면적(㎡)', '계약년월', '계약일', '가격(만원)',
                              '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시', '구']

apart_lease_deposit_only_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전월세매매구분', '전용면적(㎡)', '계약년월', '계약일',
                                       '가격(만원)', '월세(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시',
                                       '구']
apart_lease_monthly_pay_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전월세매매구분', '전용면적(㎡)', '계약년월', '계약일',
                                      '가격(만원)', '월세(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시',
                                      '구']
multiplex_lease_deposit_only_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전월세매매구분', '전용면적(㎡)', '계약년월', '계약일',
                                           '가격(만원)', '월세(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시',
                                           '구']
multiplex_lease_monthly_pay_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전월세매매구분', '전용면적(㎡)', '계약년월', '계약일',
                                          '가격(만원)', '월세(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시',
                                          '구']
officetel_lease_deposit_only_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전월세매매구분', '전용면적(㎡)', '계약년월', '계약일',
                                           '가격(만원)', '월세(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시',
                                           '구']
officetel_lease_monthly_pay_df.columns = ['시군구', '번지', '본번', '부번', '건물명', '전월세매매구분', '전용면적(㎡)', '계약년월', '계약일',
                                          '가격(만원)', '월세(만원)', '층', '건축년도', '도로명', '가격/면적', 'year', 'month', '시',
                                          '구']

In [42]:
apart_trade_df['전월세매매구분'] = '매매'
apart_trade_df['건물종류'] = '아파트'
multiplex_trade_df['전월세매매구분'] = '매매'
multiplex_trade_df['건물종류'] = '연립다세대'
officetel_trade_df['전월세매매구분'] = '매매'
officetel_trade_df['건물종류'] = '오피스텔'

apart_lease_deposit_only_df['건물종류'] = '아파트'
apart_lease_monthly_pay_df['건물종류'] = '아파트'
multiplex_lease_deposit_only_df['건물종류'] = '연립다세대' 
multiplex_lease_monthly_pay_df['건물종류'] = '연립다세대'
officetel_lease_deposit_only_df['건물종류'] = '오피스텔'
officetel_lease_monthly_pay_df['건물종류'] = '오피스텔'

In [43]:
trade_dfs_list = [apart_trade_df, multiplex_trade_df, officetel_trade_df]
lease_dfs_list = [apart_lease_deposit_only_df, apart_lease_monthly_pay_df,
                  multiplex_lease_deposit_only_df, multiplex_lease_monthly_pay_df,
                  officetel_lease_deposit_only_df, officetel_lease_monthly_pay_df]

In [44]:
def concatenate_main_data(trade_dfs, lease_dfs):
    # trade_dfs, lease_dfs are lists
    trade_dfs = trade_dfs.copy()
    lease_dfs = lease_dfs.copy()    
    
    # 대지권면적 제거
    trade_dfs[1] = trade_dfs[1].drop(columns=['대지권면적(㎡)'])
    
    # 월세 제거
    for i in range(len(lease_dfs)):
        lease_dfs[i] = lease_dfs[i].drop(columns=['월세(만원)'])
    
    #
    dfs_list = []    
    for df in trade_dfs:
        dfs_list.append(df)
        
    for df in lease_dfs:
        dfs_list.append(df)
        
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    return concat_df

In [45]:
main_data_df = concatenate_main_data(trade_dfs_list, lease_dfs_list)

In [46]:
print(main_data_df.shape)
main_data_df.head()

(4668585, 19)


,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,55000,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,매매,아파트
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,22,58250,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,매매,아파트
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201305,10,60000,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,매매,아파트


In [47]:
def read_coor_data1():
    basedir = './좌표데이터/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.txt'):
            try:
                df = pd.read_csv(basedir + filename, sep='|', header=None).drop_duplicates()
            except:
                df = pd.read_csv(basedir + filename, sep='|', header=None, encoding='ansi').drop_duplicates()
                            
            df['도로명주소'] = df[3] + ' ' + df[4] + ' ' + df[7] + ' ' + df[9].astype('str') + '-' + df[10].astype('str')
            
            df['x좌표'] = df[16]
            df['y좌표'] = df[17]
            
            df = df[['도로명주소', 'x좌표', 'y좌표']]
            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).drop_duplicates(subset=['도로명주소'], keep='last').reset_index(drop=True)
        
    return concat_df

def read_coor_data2():
    basedir = './좌표데이터2/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.txt'):
            try:
                df = pd.read_csv(basedir + filename, sep='|', header=None, usecols=[1, 2, 3, 4, 5, 7, 8, 23, 24]).drop_duplicates()
            except:
                df = pd.read_csv(basedir + filename, sep='|', header=None, encoding='ansi', usecols=[1, 2, 3, 4, 5, 7, 8, 23, 24])\
                .drop_duplicates()
                
            df['도로명주소'] = df[1] + ' ' + df[2] + ' ' + df[5] + ' ' + df[7].astype('str') + '-' + df[8].astype('str')
            
            df['x좌표'] = df[23]
            df['y좌표'] = df[24]
            
            df = df[['도로명주소', 'x좌표', 'y좌표']]
            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).drop_duplicates(subset=['도로명주소'], keep='last').reset_index(drop=True)
        
    return concat_df

def read_coor_data3():
    basedir = './좌표데이터3/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.csv'):
            df = pd.read_csv(basedir + filename, index_col=0).drop_duplicates()            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).drop_duplicates(subset=['전체주소'], keep='last').reset_index(drop=True)
    concat_df.columns = ['지번주소', 'x좌표', 'y좌표']
    
    return concat_df

def add_0(x):
    if pd.isnull(x) == False:
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x
    else:
        return x
    
def read_coor_data():
    roadname_coor_df1 = read_coor_data1()
    roadname_coor_df2 = read_coor_data2()
    
    roadname_coor_df = pd.concat([roadname_coor_df1, roadname_coor_df2]).drop_duplicates(subset=['도로명주소'], keep='last')\
    .reset_index(drop=True)
    
    landnum_coor_df = read_coor_data3()
        
    return roadname_coor_df, landnum_coor_df

In [48]:
roadname_coor_df, landnum_coor_df = read_coor_data()

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 32.01it/s]


In [49]:
print(roadname_coor_df.shape)
display(roadname_coor_df.head())
print(landnum_coor_df.shape)
display(landnum_coor_df.head())

(566869, 3)


,도로명주소,x좌표,y좌표
0,서울특별시 관악구 난곡로24가길 18-0,948851.369529,1.941318e+06
1,서울특별시 종로구 성균관로15길 33-0,955591.635372,1.954533e+06
2,서울특별시 성북구 인촌로7길 70-0,957563.072605,1.954475e+06
3,서울특별시 서초구 강남대로91길 5-0,957617.535913,1.945960e+06
4,서울특별시 서초구 바우뫼로11길 54-0,958121.727726,1.941641e+06


(90867, 3)


,지번주소,x좌표,y좌표
0,서울특별시 서초구 고무래로,956676.746259,1.945089e+06
1,서울특별시 서초구 방배로 21-0,955386.073490,1.942583e+06
2,서울특별시 성동구 난계로 9-0,957972.833890,1.951036e+06
3,서울특별시 양천구 남부순환로85길 10-0,940382.983086,1.948322e+06
4,서울특별시 은평구 응암로16길 37-0,948796.942106,1.954765e+06


In [50]:
def replace_invalid_to_nan(df):
    df = df.copy()
    
    both_0_df = df[(df['본번'] == 0) & (df['부번'] == 0)]
        
    df.loc[both_0_df.index, '번지'] = np.nan
    
    return df

In [51]:
main_data_df = replace_invalid_to_nan(main_data_df)

In [52]:
def assign_addr(df):
    df = df.copy()
    
    df['지번주소'] = df['시군구'] + ' ' + df['번지'].apply(add_0)
    df['도로명주소'] = df['시'] + ' ' + df['구'] + ' ' + df['도로명'].apply(add_0)
    
    both_nan_df = df[(df['번지'].isna()) & (df['도로명'].isna())]
    not_0_df = both_nan_df[both_nan_df['본번'] != 0]
    
    df.loc[not_0_df.index, '지번주소'] = not_0_df['시군구'] + ' ' + not_0_df['본번'].astype('str') + '-' + not_0_df['부번'].astype('str')
    
    return df

In [53]:
main_data_df = assign_addr(main_data_df)

In [54]:
def drop_no_addr(df):
    df = df.copy()
    
    no_addr_df = df[(df['지번주소'].isna()) & (df['도로명주소'].isna())]
    
    df.drop(no_addr_df.index, inplace=True)
    
    df.reset_index(drop=True, inplace=True)
    
    return df

In [55]:
main_data_df = drop_no_addr(main_data_df)

# Merging Data

In [56]:
print(main_data_df.shape)
main_data_df.head()

(4667685, 21)


,시군구,번지,본번,부번,건물명,전용면적(㎡),계약년월,계약일,가격(만원),층,건축년도,도로명,가격/면적,year,month,시,구,전월세매매구분,건물종류,지번주소,도로명주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201309,8,57000,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),77.75,201312,16,57000,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,11,55000,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201302,22,58250,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,67.28,201305,10,60000,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,매매,아파트,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0


In [66]:
main_data_df.to_csv('main_data_df_{}.csv'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")), index=False)

In [58]:
print(land_prices_df.shape)
land_prices_df.head()

(29393777, 3)


,전체주소,년,공시지가(만원)
0,서울특별시 종로구 청운동 1-0,1990,75.0
1,서울특별시 종로구 청운동 1-0,1991,98.0
2,서울특별시 종로구 청운동 1-0,1992,98.0
3,서울특별시 종로구 청운동 1-0,1993,98.0
4,서울특별시 종로구 청운동 1-0,1994,96.0


In [67]:
land_prices_df.to_csv('land_prices_df_{}.csv'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")), index=False)

In [59]:
print(land_specs_df.shape)
land_specs_df.head()

(7468497, 70)


,기준년도,전체주소,지목명_대,지목명_도로,지목명_하천,지목명_임야,지목명_공원,지목명_구거,지목명_학교용지,지목명_종교용지,지목명_사적지,지목명_주차장,지목명_철도용지,지목명_주유소용지,지목명_잡종지,지목명_제방,지목명_수도용지,지목명_전,지목명_묘지,지목명_체육용지,지목명_공장용지,지목명_창고용지,지목명_유지,지목명_답,지목명_유원지,지목명_과수원,지목명_목장용지,용도지역명1_제1종일반주거지역,용도지역명1_자연녹지지역,용도지역명1_개발제한구역,용도지역명1_제2종일반주거지역,용도지역명1_제3종일반주거지역,용도지역명1_일반상업지역,용도지역명1_준주거지역,용도지역명1_제1종전용주거지역,용도지역명1_중심상업지역,용도지역명1_보전녹지지역,용도지역명1_근린상업지역,용도지역명1_준공업지역,용도지역명1_유통상업지역,용도지역명1_제2종전용주거지역,용도지역명1_생산녹지지역,지형높이_평지,지형높이_완경사,지형높이_지정되지않음,지형높이_급경사,지형높이_고지,지형높이_저지,지형형상_사다리형,지형형상_부정형,지형형상_지정되지않음,지형형상_삼각형,지형형상_정방형,지형형상_가로장방,지형형상_역삼각형,지형형상_자루형,지형형상_세로장방,도로접면_소로한면,도로접면_지정되지않음,도로접면_맹지,도로접면_중로각지,도로접면_중로한면,도로접면_소로각지,도로접면_세로한면(가),도로접면_세로한면(불),도로접면_세로각지(불),도로접면_세로각지(가),도로접면_광대로한면,도로접면_광대세각,도로접면_광대소각
0,2013,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2014,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,2015,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,2016,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,2017,서울특별시 종로구 청운동 1-0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [68]:
land_specs_df.to_csv('land_specs_df_{}.csv'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")), index=False)